# Huffman Coding

A rudimentary program to see if I could implement the Huffman coding algorithm from scratch -- encode and decode. That's it. Can't be bothered to pretty print or optimise everything. 

Wikipedia page on **Huffman coding**: [https://en.wikipedia.org/wiki/Huffman_coding](https://en.wikipedia.org/wiki/Huffman_coding)

- **Informal description**
    - **Given**: A set of symbols and their weights (usually proportional to probabilities).

    - **Find**: A prefix-free binary code (a set of codewords) with minimum expected codeword length (equivalently, a tree with minimum weighted path length from the root).

In [546]:
from random import randint

In [547]:
# 1. Define symbols
symbols = ["A", "B", "C", "D", "E", "F"]
num_symbols = len(symbols)
weights = [0 for _ in range(num_symbols)]

In [548]:
# 2. Generate random string using the symbols
str_len = randint(20, 100) # Random string length
sym_str = "" # Total string of symbols
for i in range(0, str_len):
    sym_idx = randint(0, num_symbols - 1)
    sym_str += symbols[sym_idx]
    weights[sym_idx] += 1

weights = [w / str_len for w in weights] # Occurrence probabilities
weights_dict = {symbols[i] : round(weights[i], 2) for i in range(num_symbols)} # Mapped to the symbols

print("sym_str", sym_str)
print("weights", weights)
print("weights_dict", weights_dict)

sym_str DCFEAABEFFFBFDADECBEACBCDEEEF
weights [0.13793103448275862, 0.13793103448275862, 0.13793103448275862, 0.13793103448275862, 0.2413793103448276, 0.20689655172413793]
weights_dict {'A': 0.14, 'B': 0.14, 'C': 0.14, 'D': 0.14, 'E': 0.24, 'F': 0.21}


In [549]:
# Node class
class Node:
    def __init__(self, prob, left, right, symbol):
        self.prob = prob
        self.left = left
        self.right = right
        self.symbol = symbol

    def __str__(self):
        if self.symbol:
            return f"({self.prob}, {self.symbol})"
        else:
            return f"({self.prob})"

    def __repr__(self):
        return f"({self.left})<-({self.prob:0.2f}, {self.symbol})->({self.right})"

    def is_leaf(self):
        return not self.left and not self.right

    def print_tree(self):
        add_left = f"{self.left.print_tree()}<-" if self.left else ""
        add_right = f"->{self.right.print_tree()}" if self.right else ""

        return f"{add_left} {self} {add_right}"

    def flatten(self):
        pass # Nah

In [550]:
# 3. Convert symbols into Nodes
sym_nodes = [Node(weights[i], None, None, symbols[i]) for i in range(num_symbols)]

print(sym_nodes)

[(None)<-(0.14, A)->(None), (None)<-(0.14, B)->(None), (None)<-(0.14, C)->(None), (None)<-(0.14, D)->(None), (None)<-(0.24, E)->(None), (None)<-(0.21, F)->(None)]


In [551]:
# 4. Combine the lowest two nodes until tree is completed
while len(sym_nodes) != 1:
    # Sort by ascending probability
    sym_nodes.sort(key=lambda node: node.prob)

    # Extract lowest two nodes
    left = sym_nodes.pop(0)
    right = sym_nodes.pop(0)

    # Combine a new node and add back
    combine = Node(left.prob + right.prob, left, right, None)

    sym_nodes.append(combine) # Lazy

In [552]:
root = sym_nodes[0]
print("sym_nodes after processing:", sym_nodes) # Probability = 1.00
print(root.print_tree())  # Doesn't show depth. Just to sanity check.

sym_nodes after processing: [((0.4482758620689655))<-(1.00, None)->((0.5517241379310345))]
 (0.20689655172413793, F) <- (0.4482758620689655) -> (0.2413793103448276, E) <- (1.0) -> (0.13793103448275862, A) <- (0.27586206896551724) -> (0.13793103448275862, B) <- (0.5517241379310345) -> (0.13793103448275862, C) <- (0.27586206896551724) -> (0.13793103448275862, D) 


In [553]:
# 5. Dictionaries for symbol->code and code->symbol
codes = {}
uncodes = {}
def assign_code(codes, last_code, node : Node):
    if (node.is_leaf()):
        codes[node.symbol] = last_code
        uncodes[last_code] = node.symbol
        return
    
    if (node.left):
        assign_code(codes, last_code + "0", node.left)
    if (node.right):
        assign_code(codes, last_code + "1", node.right)

assign_code(codes, "", root)
print("weights_dict", weights_dict)
print("symbol->code", codes)
print("code->symbol", uncodes)

weights_dict {'A': 0.14, 'B': 0.14, 'C': 0.14, 'D': 0.14, 'E': 0.24, 'F': 0.21}
symbol->code {'F': '00', 'E': '01', 'A': '100', 'B': '101', 'C': '110', 'D': '111'}
code->symbol {'00': 'F', '01': 'E', '100': 'A', '101': 'B', '110': 'C', '111': 'D'}


In [554]:
# 6. Convert to string of codewords (symbol->code)
code_str =""
for c in sym_str:
    code_str += codes[c]
print("code string:", code_str)

code string: 11111000011001001010100000010100111100111011101010110011010111011101010100


In [555]:
# 7. Decode the code string into symbols (code->symbol)
curr = ""
decode = ""
for c in code_str:
    curr += c
    if (uncodes.get(curr)):
        decode += uncodes[curr]
        curr = ""
        
# Verify that the original and decoded strings are equivalent
print("original string:", sym_str)
print("decoded string:", decode)
print(f"original string =? decoded string: {sym_str==decode}")

original string: DCFEAABEFFFBFDADECBEACBCDEEEF
decoded string: DCFEAABEFFFBFDADECBEACBCDEEEF
original string =? decoded string: True
